# Demo for Predicting Fit

In [1]:
#Data cleaning
import numpy as np 
import pandas as pd

#Modeling 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

from ipywidgets import widgets, interact, interact_manual, fixed
from IPython.display import display, clear_output, HTML
from imblearn.over_sampling import SMOTE

np.random.seed(42)

#Warnings - to keep the notebook clean 
import warnings 
warnings.filterwarnings('ignore')
pd.options.display.max_columns= 5000

/Users/ariellemiro/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
#Data cleaning
import numpy as np 
import pandas as pd

#Modeling 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

from imblearn.over_sampling import SMOTE

np.random.seed(42)

#Warnings - to keep the notebook clean 
import warnings 
warnings.filterwarnings('ignore')
pd.options.display.max_columns= 5000

In [3]:
def on_predict_submit(incoming):
    
    # Insert values into dataframe
    X_input = []
    
    X_input = pd.DataFrame(columns=['age','body_type','height','rating','rented_for','size','weight', 'category'])
    
    input_variables = [input_age.value, input_body_type.value, input_height.value, input_rating.value, 
                       input_rented_for.value, input_size.value, input_weight.value, input_category.value]
    
    X_input.loc[1] = [x for x in input_variables]
    
    map_list = ['body_type', 'rented_for', 'category']
    
    X_input = map_to_numeric(X_input, columns=map_list)
    
    
    X_input = equalize_features(X_features, X_input)
    
    #the prediction
    def fit_or_not():
        if rf.predict(X_input)[0] == 0:
            return 'Too Small'
        elif rf.predict(X_input)[0] == 1:
            return 'Fits true to size!'
        elif rf.predict(X_input)[0] == 2:
            return 'Too Large'
    print(f"Does the garment fit?: {fit_or_not()}")

In [4]:
# Input age
input_age = widgets.FloatSlider(
    value=0,
    min=15,
    max=91,
    step=1,
    description='Age',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(input_age)


# Input rating
input_rating = widgets.FloatSlider(
    value=0,
    min=2,
    max=10,
    step=1,
    description='Rating',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(input_rating)


# Input size
input_size = widgets.FloatSlider(
    value=0,
    min=0,
    max=58,
    step=1,
    description='Size',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(input_size)


# Input height
input_height = widgets.FloatSlider(
    value=0,
    min=54,
    max=76,
    step=0.5,
    description='Height',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(input_height)


# Input weight
input_weight = widgets.FloatSlider(
    value=0,
    min=50,
    max=285,
    step=0.5,
    description='Weight',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(input_weight)


# Input body type
body_type = ['hourglass', 'athletic', 'petite', 'pear', 'straight & narrow', 'full bust', 'apple']
input_body_type = widgets.ToggleButtons(
    options=body_type,
    description='Body Type',
    disabled=False,
    button_style='',
    tooltips=body_type
)
display(input_body_type)

# Input rented for
rented_for = ['wedding', 'formal affair', 'party', 'everyday', 'work', 'other', 'date', 'vacation']
input_rented_for = widgets.ToggleButtons(
    options=rented_for,
    description='Rented For',
    disabled=False,
    button_style='',
    tooltips=rented_for
)
display(input_rented_for)


# Input category
category = ['dress', 'gown', 'sheath', 'jumpsuit', 'shift', 'top', 'maxi', 'romper', 'jacket', 'mini', 'skirt',
            'sweater', 'coat', 'blazer', 'shirtdress', 'blouse', 'down', 'pants', 'shirt', 'vest', 'culottes',
            'leggings', 'cardigan', 'frock', 'tunic', 'bomber', 'tank', 'suit', 'sweatshirt', 'print', 'cape',
            'trousers', 'pullover', 'midi', 'tee', 'peacoat', 'poncho', 'kimono', 'turtleneck', 'parka', 'tight',
            'knit', 'duster', 'kaftan', 'hoodie', 'overalls', 'crewneck', 'sweatpants', 'henley', 'jogger',
            'trench', 'cami']
input_category = widgets.ToggleButtons(
    options=category,
    description='Category',
    disabled=False,
    button_style='',
    tooltips=category
)
display(input_category)


# Input Predict
input_predict = widgets.Button(
    value=False,
    description='Predict',
    disabled=False,
    button_style='',
    tooltip='Predict',
    icon='check'
)
display(input_predict)
input_predict.on_click(on_predict_submit)

FloatSlider(value=15.0, continuous_update=False, description='Age', max=91.0, min=15.0, readout_format='.1f', …

FloatSlider(value=2.0, continuous_update=False, description='Rating', max=10.0, min=2.0, readout_format='.1f',…

FloatSlider(value=0.0, continuous_update=False, description='Size', max=58.0, readout_format='.1f', step=1.0)

FloatSlider(value=54.0, continuous_update=False, description='Height', max=76.0, min=54.0, readout_format='.1f…

FloatSlider(value=50.0, continuous_update=False, description='Weight', max=285.0, min=50.0, readout_format='.1…

ToggleButtons(description='Body Type', options=('hourglass', 'athletic', 'petite', 'pear', 'straight & narrow'…

ToggleButtons(description='Rented For', options=('wedding', 'formal affair', 'party', 'everyday', 'work', 'oth…

ToggleButtons(description='Category', options=('dress', 'gown', 'sheath', 'jumpsuit', 'shift', 'top', 'maxi', …

Button(description='Predict', icon='check', style=ButtonStyle(), tooltip='Predict')

Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Fits true to size!
Does the garment fit?: Too Large


In [5]:
def equalize_features(model_features, new_df):
    new_features = new_df.columns.tolist()
    if set(model_features) != set(new_features):
        missing_cols = set(model_features) - set(new_features)
        for col in missing_cols:
            new_df[col] = 0
    
    if set(new_features) != set(model_features):
        missing_cols = set(new_features) - set(model_features)
        new_df = new_df.drop(missing_cols, axis=1)
    
    new_df = new_df[model_features]
    return new_df

In [6]:
# Map values to numeric
def map_to_numeric(X_input, columns):
    for column in columns:
        map_dict = get_map_dict(column)
        X_input[column] = X_input[column].map(map_dict)
    return X_input
    
def get_map_dict(feature):
    map_feature = {
    'body_type': {
        'hourglass': 1, 
        'athletic': 2, 
        'petite': 3, 
        'pear': 4, 
        'straight & narrow': 5, 
        'full bust': 6, 
        'apple': 7}, 
    'rented_for': {
        'wedding': 1,
        'formal affair': 2, 
        'party': 3, 
        'everyday': 4, 
        'work': 5, 
        'other': 6, 
        'date': 7, 
        'vacation': 8},
    'category': {
        'dress': 1, 
        'gown': 2, 
        'sheath': 3, 
        'jumpsuit': 4, 
        'shift': 5, 
        'top': 6, 
        'maxi': 7, 
        'romper': 8, 
        'jacket': 9,
        'mini': 10,
        'skirt': 11,
        'sweater': 12, 
        'coat': 13, 
        'blazer': 14, 
        'shirtdress': 15, 
        'blouse': 16, 
        'down': 17, 
        'pants': 18, 
        'shirt': 19, 
        'vest': 20, 
        'culottes': 21,
        'leggings': 22, 
        'cardigan': 23, 
        'frock': 24, 
        'tunic': 25, 
        'bomber': 26, 
        'tank': 27, 
        'suit': 28, 
        'sweatshirt': 29, 
        'print': 30, 
        'cape': 31,
        'trousers': 32, 
        'pullover': 33, 
        'midi': 34, 
        'tee': 35, 
        'peacoat': 36, 
        'poncho': 37, 
        'kimono': 38, 
        'turtleneck': 39, 
        'parka': 40, 
        'tight': 41,
        'knit': 42, 
        'duster': 43, 
        'kaftan': 44,
        'hoodie': 45, 
        'overalls': 46, 
        'crewneck': 47, 
        'sweatpants': 48, 
        'henley': 49, 
        'jogger': 50,
        'trench': 51,
        'cami': 52}
    }
    return map_feature[feature]

In [7]:
# Function to run different model types

def fit__and_evaluate(X, y, model_name='lr', model_type=LogisticRegression()):
    
    # Pipeline for pre-processing
    pipe = Pipeline([
        ('scale', StandardScaler()),
        (model_name, model_type)
    ])
    # Fit the model 
    model = pipe.fit(X, y)
    
    #Score
    score = model.score(X, y)
                   
    # Print attributes
    print(f"For model: {model_type}")
    print(f"Score: {score}")
    
    return model

In [8]:

def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Evaluate training data
    preds_train = model.predict(X_train)
    kf_train = KFold(shuffle=True, random_state=42)
    cvs_train = cross_val_score(model, X_train, y_train, cv=kf_train).mean()
    score_train = model.score(X_train, y_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))

    # Evaluate testing data
    preds_test = model.predict(X_test)
    kf_test = KFold(shuffle=True, random_state=42)
    cvs_test = cross_val_score(model, X_test, y_test, cv=kf_test).mean()
    score_test = model.score(X_test, y_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, preds_test))
    
    # Print results
    print(f'cvs mean score on training data: {cvs_train:.4f}')
    print(f'cvs mean score on testing data: {cvs_test:.4f}')
    
    print(f'Train Score: {score_train:.4f}')
    print(f'Test Score: {score_test:.4f}')
    
    print(f'Train RMSE: {rmse_train:.4f}')
    print(f'Test RMSE: {rmse_test:.4f}')


# Model

In [9]:
df = pd.read_csv('./rent_the_runway_cleaned_with_dummies.csv')
df.columns

Index(['age', 'bust_size', 'fit', 'height', 'item_id', 'rating', 'review_date',
       'review_summary', 'review_text', 'size', 'user_id', 'weight',
       'body_type_apple', 'body_type_athletic', 'body_type_full_bust',
       'body_type_hourglass', 'body_type_pear', 'body_type_petite',
       'body_type_straight_narrow', 'rented_for_date', 'rented_for_everyday',
       'rented_for_formal_affair', 'rented_for_other', 'rented_for_party',
       'rented_for_vacation', 'rented_for_wedding', 'rented_for_work',
       'category_blazer', 'category_blouse', 'category_bomber',
       'category_cami', 'category_cape', 'category_cardigan', 'category_coat',
       'category_crewneck', 'category_culottes', 'category_down',
       'category_dress', 'category_duster', 'category_frock', 'category_gown',
       'category_henley', 'category_hoodie', 'category_jacket',
       'category_jogger', 'category_jumpsuit', 'category_kaftan',
       'category_kimono', 'category_knit', 'category_leggings',
      

In [10]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).drop(columns= 'fit')

In [11]:
X = numeric_cols
y = df['fit']

In [12]:
#Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =42)

In [13]:
# sm_ = SMOTE(sampling_strategy= 1, kind ='regular', random_state=42)
# X_resample, y_resample = sm.fit_sample(X, y.ravel())

In [14]:
# from imblearn.over_sampling import SMOTE

# print("Before OverSampling, counts of label '2': {}".format(sum(y_train==2)))
# print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
# print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

# sm = SMOTE(random_state=42)

# X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

# print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

# print("After OverSampling, counts of label '2': {}".format(sum(y_train_res==2)))
# print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
# print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

In [16]:
rf = fit__and_evaluate(X_train, y_train, model_name='rf', model_type=RandomForestClassifier())

For model: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Score: 0.9808139269926628


In [18]:
evaluate_model(rf, X_train, X_test, y_train, y_test)

cvs mean score on training data: 0.7116
cvs mean score on testing data: 0.7100
Train Score: 0.9808
Test Score: 0.7070
Train RMSE: 0.1434
Test RMSE: 0.5772


In [19]:
X_features = X.columns.tolist()